# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [19]:
import os
import pandas as pd


data_dir = os.path.abspath('../../05_src/data/adult/')
data_file = os.path.join(data_dir, 'adult.data')
test_file = os.path.join(data_dir, 'adult.test')


print("Data directory:", data_dir)
print("Data file path:", data_file)
print("Test file path:", test_file)




columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]


adult_dt = (pd.read_csv(data_file, header=None, names=columns)
              .assign(income=lambda x: (x.income.str.strip() == '>50K')*1))

print("Data loaded successfully!")



Data directory: c:\Users\User\Desktop\scaling_to_production\05_src\data\adult
Data file path: c:\Users\User\Desktop\scaling_to_production\05_src\data\adult\adult.data
Test file path: c:\Users\User\Desktop\scaling_to_production\05_src\data\adult\adult.test
Data loaded successfully!


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [36]:

X = adult_dt.drop('income', axis=1)


Y = adult_dt['income']


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The data split's reproducibility is guaranteed by its random state. Consistent outcomes are possible by setting it to a fixed number (such as 42), which guarantees that the same train-test split is generated each time. 

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [37]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier


numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
])


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  
    ('classifier', RandomForestClassifier())
])


model_pipeline.fit(X_train, Y_train)


Y_pred = model_pipeline.predict(X_test)


accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")



Accuracy: 0.86


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [39]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': make_scorer(roc_auc_score),
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True)
}


cv_results = cross_validate(model_pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)


print("Training Scores:")
print(f"Accuracy: {cv_results['train_accuracy'].mean():.3f}")
print(f"Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.3f}")
print(f"ROC AUC: {cv_results['train_roc_auc'].mean():.3f}")
print(f"Negative Log Loss: {cv_results['train_neg_log_loss'].mean():.3f}")

print("\nValidation Scores:")
print(f"Accuracy: {cv_results['test_accuracy'].mean():.3f}")
print(f"Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.3f}")
print(f"ROC AUC: {cv_results['test_roc_auc'].mean():.3f}")
print(f"Negative Log Loss: {cv_results['test_neg_log_loss'].mean():.3f}")


c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\User\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Scores:
Accuracy: 1.000
Balanced Accuracy: 1.000
ROC AUC: 1.000
Negative Log Loss: -0.080

Validation Scores:
Accuracy: 0.854
Balanced Accuracy: 0.775
ROC AUC: 0.775
Negative Log Loss: -0.369


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [40]:
import pandas as pd


results_dict = {
    'Fold': range(1, 6),
    'Train Accuracy': cv_results['train_accuracy'],
    'Train Balanced Accuracy': cv_results['train_balanced_accuracy'],
    'Train ROC AUC': cv_results['train_roc_auc'],
    'Train Neg Log Loss': cv_results['train_neg_log_loss'],
    'Test Accuracy': cv_results['test_accuracy'],
    'Test Balanced Accuracy': cv_results['test_balanced_accuracy'],
    'Test ROC AUC': cv_results['test_roc_auc'],
    'Test Neg Log Loss': cv_results['test_neg_log_loss']
}

results_df = pd.DataFrame(results_dict)


sorted_results_df = results_df.sort_values(by='Test Neg Log Loss', ascending=True)


print(sorted_results_df)


   Fold  Train Accuracy  Train Balanced Accuracy  Train ROC AUC  \
3     4        0.999962                 0.999920       0.999920   
1     2        0.999962                 0.999920       0.999920   
4     5        0.999885                 0.999761       0.999761   
0     1        0.999962                 0.999920       0.999920   
2     3        0.999885                 0.999761       0.999761   

   Train Neg Log Loss  Test Accuracy  Test Balanced Accuracy  Test ROC AUC  \
3           -0.080735       0.861179                0.779453      0.779453   
1           -0.079779       0.851044                0.769730      0.769730   
4           -0.080963       0.857494                0.779639      0.779639   
0           -0.080235       0.851067                0.766373      0.766373   
2           -0.080615       0.851351                0.777336      0.777336   

   Test Neg Log Loss  
3          -0.385235  
1          -0.384355  
4          -0.360361  
0          -0.358507  
2          -0

Calculate the mean of each metric. 

In [41]:

mean_metrics = sorted_results_df.mean()


print("Mean Metrics:")
print(mean_metrics)

Mean Metrics:
Fold                       3.000000
Train Accuracy             0.999931
Train Balanced Accuracy    0.999857
Train ROC AUC              0.999857
Train Neg Log Loss        -0.080465
Test Accuracy              0.854427
Test Balanced Accuracy     0.774506
Test ROC AUC               0.774506
Test Neg Log Loss         -0.368603
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [42]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss


Y_pred_proba = model_pipeline.predict_proba(X_test)


Y_pred_proba_pos = Y_pred_proba[:, 1]


accuracy = accuracy_score(Y_test, Y_pred)
balanced_accuracy = balanced_accuracy_score(Y_test, Y_pred)
roc_auc = roc_auc_score(Y_test, Y_pred_proba_pos)
neg_log_loss = log_loss(Y_test, Y_pred_proba)


metrics_results = {
    'Accuracy': accuracy,
    'Balanced Accuracy': balanced_accuracy,
    'ROC AUC': roc_auc,
    'Negative Log Loss': neg_log_loss
}


print(metrics_results)



{'Accuracy': 0.8560753403623708, 'Balanced Accuracy': 0.7757667874142, 'ROC AUC': 0.9015322995304005, 'Negative Log Loss': 0.3635128915581467}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding the target variable income immediately after loading the data streamlines the preparation process, enhances code clarity and reduces potential errors.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.